<a href="https://colab.research.google.com/github/dark-matter08/scrappers/blob/main/Canondale_bikes_scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import html5lib
from google.colab import drive
import json
drive.mount('/content/drive')
from lxml import html

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
categories_url = [
    {
        'category': 'Road',
        'sub_cat': [
            {'name': 'Race', 'link': 'https://www.cannondale.com/en-us/bikes/road/race?priceMin=1010&priceMax=13050'},
            {'name': 'Edurance', 'link': 'https://www.cannondale.com/en-us/bikes/road/endurance?priceMin=1325&priceMax=9050'},
            {'name': 'Gravel', 'link': 'https://www.cannondale.com/en-us/bikes/road/gravel?priceMin=1325&priceMax=10000'},
            {'name': 'Cyclocross', 'link': 'https://www.cannondale.com/en-us/bikes/road/cyclocross?priceMin=1875&priceMax=7050'},
            {'name': 'Electric', 'link': 'https://www.cannondale.com/en-us/bikes/electric/e-road?priceMin=3625&priceMax=10000'},
        ]
    },
    {
        'category': 'Mountain',
        'sub_cat': [
            {'name': 'Cross Country', 'link': 'https://www.cannondale.com/en-us/bikes/mountain/cross-country?priceMin=2225&priceMax=13000'},
            {'name': 'Trail', 'link': 'https://www.cannondale.com/en-us/bikes/mountain/trail-bikes?priceMin=645&priceMax=8300'},
            {'name': 'Mountain E-Bikes', 'link': 'https://www.cannondale.com/en-us/bikes/electric/e-mountain?priceMin=4550&priceMax=8650'},
        ]
    },
    {
        'category': 'Active',
        'sub_cat': [
            {'name': 'Fitness', 'link': 'https://www.cannondale.com/en-us/bikes/active/fitness?priceMin=543&priceMax=2975'},
            {'name': 'Urban', 'link': 'https://www.cannondale.com/en-us/bikes/active/urban?priceMin=1275&priceMax=5500'},
            {'name': 'Fitness E-Bikes', 'link': 'https://www.cannondale.com/en-us/bikes/electric/e-fitness?priceMin=1925&priceMax=2975'},
            {'name': 'Touring E-Bikes', 'link': 'https://www.cannondale.com/en-us/bikes/electric/e-touring?priceMin=2825&priceMax=4650'},
            {'name': 'Urban E-Bikes', 'link': 'https://www.cannondale.com/en-us/bikes/electric/e-urban?priceMin=2875&priceMax=5500'}
        ]
    },
    {
        'category': 'Kids',
        'sub_cat': [
            {'name': 'Ages 1-4 (12" wheel)', 'link': 'https://www.cannondale.com/en-us/bikes/kids/1-to-4?priceMin=195&priceMax=315'},
            {'name': 'Ages 4-6 (16" wheel)', 'link': 'https://www.cannondale.com/en-us/bikes/kids/4-to-6?priceMin=280&priceMax=350'},
            {'name': 'Ages 5-8 (20" wheel)', 'link': 'https://www.cannondale.com/en-us/bikes/kids/5-to-8?priceMin=310&priceMax=810'},
            {'name': 'Ages 7+ (24"+ wheel)', 'link': 'https://www.cannondale.com/en-us/bikes/kids/7-to-12?priceMin=415&priceMax=840'}
        ]
    }
]

In [21]:
categories_csv = [
    {
        'category': 'Road',
        'sub_cat': [
            {'name': 'Race', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-race-bikes.csv'},
            {'name': 'Edurance', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-race-endurance.csv'},
            {'name': 'Gravel', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-road-gravel.csv'},
            {'name': 'Cyclocross', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-road-cyclocross.csv'},
            {'name': 'Electric', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-road-electric.csv'},
        ]
    },
    {
        'category': 'Mountain',
        'sub_cat': [
            {'name': 'Cross Country', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-mountain-cross-country.csv'},
            {'name': 'Trail', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-mountain-trail.csv'},
            {'name': 'Mountain E-Bikes', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-mountain-electric.csv'},
        ]
    },
    {
        'category': 'Active',
        'sub_cat': [
            {'name': 'Fitness', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-active-fitness.csv'},
            {'name': 'Urban', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-active-urban.csv'},
            {'name': 'Fitness E-Bikes', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-active-electric-fitness.csv'},
            {'name': 'Touring E-Bikes', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-active-electric-touring.csv'},
            {'name': 'Urban E-Bikes', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-active-electric-urban.csv'}
        ]
    },
    {
        'category': 'Kids',
        'sub_cat': [
            {'name': 'Ages 1-4 (12" wheel)', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-kids-Ages-1-4.csv'},
            {'name': 'Ages 4-6 (16" wheel)', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-kids-Ages-4-6.csv'},
            {'name': 'Ages 5-8 (20" wheel)', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-kids-Ages-5-8.csv'},
            {'name': 'Ages 7+ (24"+ wheel)', 'link': '/content/drive/MyDrive/Colab Notebooks/cannondale/cannondale-kids-Ages-7-13.csv'}
        ]
    }
]

In [5]:
headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
}

In [ ]:
links = []
all_categories = []
sub_categories = []

for category in categories_url:
  category_name = category['category']
  sub_items = category['sub_cat']

  for item in sub_items:
    sub_cat_name = item['name']
    link = item['link']
    print(link)

    link_response = requests.get(link)
    soup = BeautifulSoup(link_response.content, 'html.parser')
    tree = html.fromstring(link_response.text)
    data = tree.xpath('div')
    bikes = soup.find_all("div", class_="card-inner")
    print(bikes)
    for bike in bikes:
      try:
        bike_link = bike.find('a', {'class': 'content product product-card__link'})
        bike_link = bike.get('href')
        links.append(bike_link)
        all_categories.append(category_name)
        sub_categories.append(sub_cat_name)
      except Exception as e:
        print(f"======{e}======")

    print(f"============================================={category_name}: {sub_cat_name}=======================================================") 

print(links)
print(len(links))

In [30]:
links = []
all_categories = []
sub_categories = []

for category in categories_csv:
  category_name = category['category']
  sub_items = category['sub_cat']

  for item in sub_items:
    sub_cat_name = item['name']
    csv_link = item['link']
    data = pd.read_csv(csv_link)
    try:
      link_list = data['content href']
      link_list = link_list.tolist()
      for link in link_list:
        links.append(link)
      print(f"======================{category_name}: {sub_cat_name}==========================")
    except Exception as e:
      print(e)


print(links)
print(len(links))

======================Road: Race==========================
======================Road: Edurance==========================
======================Road: Gravel==========================
======================Road: Cyclocross==========================
======================Road: Electric==========================
======================Mountain: Cross Country==========================
======================Mountain: Trail==========================
======================Mountain: Mountain E-Bikes==========================
======================Active: Fitness==========================
======================Active: Urban==========================
======================Active: Fitness E-Bikes==========================
======================Active: Touring E-Bikes==========================
======================Active: Urban E-Bikes==========================
======================Kids: Ages 1-4 (12" wheel)==========================
======================Kids: Ages 4-6 (16" wheel)===============

In [76]:
names = []
prices = []
description_list = []
main_image_list = []
image2_list = []
overview_section_list = []
tech_section_list = []
specs_section_list = []
geometry_section_list = []
count = 1

for link in links[:1]:
  print(f"================================== Link Count: {count}================================================")
  print(link)
  link_response = requests.get(link, headers=headers)
  link_soup = BeautifulSoup(link_response.content, 'html.parser')

  name1 = str(link_soup.find('h1', {'class': 'headline bike-configuration__headline'}).find('strong', {'class': 'platform'}).text).strip()
  name2 = str(link_soup.find('h1', {'class': 'headline bike-configuration__headline'}).text).replace(name1, '').strip()
  name = f"{name1} {name2}"
  names.append(name)

  # price = str(link_soup.find('div', {'class': 'bike-configuration__price'}).text).replace(',', '').replace('$', '').strip()
  # price = str(link_soup.find_all('script')[16].text).strip().replace('dataLayer.push({ ecommerce: null });', '')
  # price = price.replace('dataLayer.push({', '')
  # price = price.replace('event: "view_item",', '')

  bike_data = link_soup.find('script', {'type':'application/ld+json'}).text
  bike_data = json.loads(bike_data)

  print(type(bike_data))

  count = count + 1

================================== Link Count: 1================================================
https://www.cannondale.com/en-us/bikes/road/race/systemsix/systemsix-hi-mod-dura-ace-di2-c11131u
<class 'dict'>
